In [ ]:
from Data_Prep import Data_Prep, Player_IO
import torch
import Prep_Map
import Output_Map
from Player_Dataset import Create_Test_Train_Datasets
import Model_Train
from Player_Model import RNN_Model
from tqdm.notebook import tqdm
from Constants import device
from pylab import savefig
from matplotlib import pyplot as plt
from Constants import DEFAULT_HIDDEN_SIZE_HITTER, DEFAULT_NUM_LAYERS_HITTER
import numpy as np
from pathlib import Path

output_idx = 3
element_name = Model_Train.ELEMENT_LIST[output_idx]
filename = element_name + "Arch"

outfile_idx = 0
while True:
    outfile = f'Experiments/{filename}_LR_{outfile_idx}.png'
    outfile_path = Path(outfile)
    if not outfile_path.exists():
        break
    outfile_idx += 1

batch_size = 800
xs = np.linspace(0.001, 0.05, 50)
ys = []
title = element_name + " Loss vs Learning Rate"

x_label = "Learning Rate"
y_label = "Test Loss"

data_prep = Data_Prep(Prep_Map.base_prep_map, Output_Map.base_output_map)
hitter_io_list = data_prep.Generate_IO_Hitters("WHERE lastMLBSeason<? AND signingYear<? AND isHitter=?", (2025,2015,1), use_cutoff=True)
train_dataset, test_dataset = Create_Test_Train_Datasets(hitter_io_list, 0.25, 0)

num_layers = DEFAULT_NUM_LAYERS_HITTER
hidden_size = DEFAULT_HIDDEN_SIZE_HITTER
hitting_mutators = data_prep.Generate_Hitting_Mutators(batch_size, Player_IO.GetMaxLength(hitter_io_list))

training_generator = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
testing_generator = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

with tqdm(total=len(xs), desc="Total Training Runs") as pbar:
    for x in xs:
        network = RNN_Model(train_dataset.get_input_size(), 
                            num_layers, 
                            hidden_size, 
                            hitting_mutators, 
                            data_prep=data_prep, 
                            is_hitter=True)
        
        network.optimizer.param_groups[output_idx + 1]['lr'] = x
        network = network.to(device)
        end_loss = Model_Train.trainAndGraph(network, 
                                                train_dataset,
                                                test_dataset,
                                                batch_size=batch_size,
                                                num_epochs=41, 
                                                logging_interval=10000, 
                                                early_stopping_cutoff=40, 
                                                should_output=False, 
                                                model_name=f"Models/exp",
                                                save_last=False,
                                                elements_to_save=[output_idx],
                                                get_end_loss=True)
        ys.append(end_loss[output_idx])
        pbar.update(1)

In [ ]:
plt.scatter(xs, ys)
plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title(title)
plt.tight_layout(pad=0.25)
plt.savefig(outfile, dpi=400)